In [2]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

In [94]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

In [80]:
split_word = [x.split(' ') for x in list(train_data['text'])]
split_word_test = [x.split(' ') for x in list(test_data['text'])]
train_data['text_split'] = split_word
test_data['text_split'] = split_word_test

In [77]:
train, valid = train_test_split(train_data, test_size = 0.2, random_state = 42) 

In [82]:
train.to_csv('data/train_use.csv', index=False)
valid.to_csv('data/valid_use.csv', index=False)
test_data.to_csv('data/test_use.csv', index=False)

## Sklearn 

In [4]:
train = pd.read_csv("data/train_use.csv")
valid = pd.read_csv("data/valid_use.csv")
test = pd.read_csv("data/test_use.csv")

In [96]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10000)

In [97]:
X_raw = [str(train['keyword'][i]) + ' ' + train['text'][i] for i in range(len(train['keyword']))]
X_raw_valid = [str(valid['keyword'][i]) + ' ' + valid['text'][i] for i in range(len(valid['keyword']))]
X_raw_test = [str(test['keyword'][i]) + ' ' + test['text'][i] for i in range(len(test['keyword']))]
print(len(X_raw))
print(len(X_raw_valid))
print(len(X_raw_test))
print(len(X_raw) + len(X_raw_valid) + len(X_raw_test))

6090
1523
3263
10876


In [98]:
X_whole = X_raw.copy()
X_whole.extend(X_raw_valid)
X_whole.extend(X_raw_test)
print(len(X_whole))

10876


In [99]:
X = hv.transform(X_whole).toarray()
X.shape

(10876, 10000)

In [100]:
X_train = X[:6090]
X_valid = X[6090:(6090+1523)]
X_test = X[(6090+1523):]

In [101]:
y_train = list(train['target'])
y_valid = list(valid['target'])

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
lr = LogisticRegression(random_state=0).fit(X_train, y_train)

In [104]:
lr.score(X_valid, y_valid)

0.7912015758371634

In [105]:
lr_predict = lr.predict(X_test)
len(lr_predict)

3263

In [106]:
preds_df = pd.DataFrame({"id": test_data["id"], "target": lr_predict})
preds_df.to_csv(f"data/output_lr_10000.csv", index=False)

## RNN

In [7]:
from torchtext.data import Field, TabularDataset, BucketIterator

ImportError: cannot import name 'Field' from 'torchtext.data' (/workspaces/kaggle-nlp-disasters/.venv/lib/python3.8/site-packages/torchtext/data/__init__.py)

In [9]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns